In [9]:
from __future__ import print_function
from yql.api.v1.client import YqlClient

In [10]:
!git clone https://github.yandex-team.ru/edvls/statbox-abt-metrics.git
from statbox_abt_metrics.api import run
from statbox_abt_metrics.metrics.metrics_group import (
    metrics_group, YTMetricsGroup
)

fatal: destination path 'statbox-abt-metrics' already exists and is not an empty directory.


In [11]:
import statbox_abt_metrics

In [12]:
users_table = "//home/mobilesearch/personal_homes/bobsmirnov/experiments-23328/exp_uuids_full"
start_date = "2018-09-04"
end_date = "2018-09-24"
control_id = "93016"
experiment_id = "93017"

YQL_TOKEN = "XXXXX" # Open https://yql.yandex-team.ru/?settings_mode=token in browser to get your token
tmp_sessions_floder = "tmp/edvls_sessions_copy" 

In [16]:
yql_request = '''USE hahn;
PRAGMA yt.InferSchema;
PRAGMA SimpleColumns;

$tasks_folder = "home/voice/dialog/sessions";
$results_folder = "{tmp_floder}";

DEFINE ACTION $process_task($input_path) AS
    $output_path = String::ReplaceAll($input_path, $tasks_folder, $results_folder);
    
    INSERT INTO $output_path WITH TRUNCATE
    SELECT 
        YQL::Append(Yson::ConvertToStringList(sessions.testids), CAST(users.test_id AS String)) AS testids,
        sessions.* WITHOUT sessions.testids
    FROM $input_path AS sessions 
    JOIN [{users_table}] AS users
    ON  String::SplitToList(sessions.uuid, "/"){{1}} == users.uuid
    WHERE users.exp_start <= sessions.fielddate 
    ;
END DEFINE;

$tasks = (
    SELECT LIST(Path)
    FROM FOLDER($tasks_folder)
    WHERE Type == "table"
    AND Path >= $tasks_folder || "/{start_date}"
    AND Path <= $tasks_folder || "/{end_date}"
);

EVALUATE FOR $task IN $tasks ?? ListCreate(TypeOf($tasks)) DO $process_task($task);
'''.format(tmp_floder=tmp_sessions_floder, start_date=start_date, end_date=end_date, users_table=users_table)

In [ ]:
client = YqlClient(token=YQL_TOKEN)
request = client.query(yql_request)
request.run()
request.get_results()

In [ ]:
SESSION_LOGS = tmp_sessions_floder

In [ ]:
from statbox_abt_metrics.metrics.alice.basic_scenarios_metrics import *

@metrics_group
class BasicScenariosMetricsModified(YTMetricsGroup):
    reducer = IntentMetrics()

    name = "basic_scenarios_metrics"

    ru_name = "Базовые метрики сценариев"

    en_name = "Basic scenarios metrics"

    def build_meta(self):
        return make_metrics_meta()

    def build_stream(self, job):
        return job \
            .log(SESSION_LOGS) \
            .project(
                qe.all(),
                unfold_experiments(self.config.experiments),
                user_id='uuid'
            ) \
            .call(
                unfold_datetime,
                self.config,
                qe.log_field('fielddate')
            ) \
            .groupby('datetime', 'user_id', 'exp_id') \
            .reduce(self.reducer)

In [ ]:
from statbox_abt_metrics.metrics.alice.basic_metrics import *

@metrics_group
class BasicMetricsModified(YTMetricsGroup):
    name = "basic_metrics"
    ru_name = "Базовые метрики"
    en_name = "Basic metrics"

    reducer = None

    def init(self):
        self.reducer = BasicMetricsCounter()

    def build_meta(self):
        return self.reducer.meta

    def build_stream(self, job):
        return job \
            .log(SESSION_LOGS) \
            .project(
                qe.all(),
                unfold_experiments(self.config.experiments),
                user_id='uuid'
            ) \
            .call(
                unfold_datetime,
                self.config,
                qe.log_field('fielddate')
            ) \
            .groupby('datetime', 'user_id', 'exp_id') \
            .reduce(self.reducer)

In [ ]:
from statbox_abt_metrics.metrics.alice.session_metrics import *

@metrics_group
class SessionMetricsModified(YTMetricsGroup):
    name = "session_metrics"
    ru_name = "Сессионные метрики"
    en_name = "Sessions metrics"

    reducer = None

    def init(self):
        self.reducer = AliceSessionsCounterReduce()

    def build_meta(self):
        return self.reducer.meta

    def build_stream(self, job):
        return job.log(
            SESSION_LOGS
        ).project(
            qe.all(),
            unfold_experiments(self.config.experiments),
            user_id='uuid',
        ).groupby(
            'fielddate', 'user_id', 'exp_id',
        ).sort(
            'session_id',
        ).reduce(
            unfold_sessions,
        ).call(
            unfold_datetime,
            self.config,
            qe.log_field('fielddate'),
        ).groupby(
            'datetime', 'user_id', 'exp_id',
        ).sort(
            'timestamp',
        ).reduce(
            self.reducer,
        )

In [ ]:
from statbox_abt_metrics.metrics.alice.retention_metrics import *

@metrics_group
class RetentionMetricsModified(AliceMetricsGroup):
    name = "retention_metrics"
    ru_name = "Метрики возвращаемость"
    en_name = "Retention metrics"

    reducer = None

    def init(self):
        self.reducer = RetentionMetricsCounter(
            self.config.start_date,
            self.config.end_date
        )

    def build_meta(self):
        return self.reducer.meta

    def build_stream(self, job):
        return job.log(
            SESSION_LOGS
        ).project(
            qe.all(),
            self.unfold_experiments(),
            user_id='uuid'
        ).call(
            unfold_datetime,
            self.config,
            qe.log_field('fielddate')
        ).groupby(
            'user_id', 'exp_id'
        ).reduce(self.reducer)


In [ ]:
result = run(
    service='transport',
    metric_groups=[BasicMetricsModified, SessionMetricsModified, BasicScenariosMetricsModified, RetentionMetricsModified],
    experiments=[control_id, experiment_id],
    start_date=start_date,
    end_date=end_date
)

In [ ]:
result